### BOJ 문제 목록 크롤링

백준 온라인 저지[https://www.acmicpc.net/](https://www.acmicpc.net/)에 존재하는 약 30,000개의 문제 정보를 크롤링하는 코드입니다.  
크롤링할 대상은 아래와 같습니다.  크롤링하기 위해서는 selenium 패키지 설치가 필요하며, `pip install selenium` 을 통해 바로 설치할 수 있습니다.

1. (번호) 문제 번호
2. (제출) 문제에 제출된 답안의 횟수
3. (정답) 정답을 맞춘 답안의 수
4. (맞힌 사람) 문제를 맞춘 사람의 수
5. (정답 비율) 문제의 정답률
6. (난이도) Solved.ac에서 제공하는 문제의 난이도

In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from tqdm import tqdm
import pandas as pd

In [2]:
# 크롬드라이버 옵션 (창 안보이게 숨기기)
driver_option = webdriver.ChromeOptions()
# driver_option.add_argument('headless')

### [1] 문제 정보 크롤링 (5가지 feature)

- problemId
- problemTitle
- solved
- submissionCount
- correcRatio

In [ ]:
# 크롬드라이버 실행
driver = webdriver.Chrome(options=driver_option) 
driver.implicitly_wait(3)

interaction = []

driver.get('https://www.acmicpc.net/school/ranklist/187')
time.sleep(3)

page_elem = driver.find_elements(by=By.XPATH, value="(//ul[@class='pagination']/li/a)[position()>1]")
MAX_PAGE = max([int(elem.get_attribute('href').split("/")[-1]) for elem in page_elem])


for page in tqdm(range(MAX_PAGE)):
    driver.get(f'https://www.acmicpc.net/school/ranklist/187/{page + 1}')

    # 페이지가 완전히 로딩되도록 3초동안 기다림
    time.sleep(3)

    gachon_user_elem = driver.find_elements(by=By.CSS_SELECTOR, value='#ranklist > tbody > tr > td:nth-child(2) > a')
    user_list = [user_elem.get_attribute('href').split("/")[-1] for user_elem in gachon_user_elem] # 현재 page의 가천대학교 학생 목록

    pbar = tqdm(user_list, leave=False)
    for user_name in pbar:
        driver.get(f'https://www.acmicpc.net/user/{user_name}')  # 백준 유저 정보 페이지 접속
        time.sleep(3)

        # -- 유저가 해결한 문제
        solved_problems = driver.find_elements(by=By.XPATH, value="//div[contains(@class, 'panel') and contains(@class, 'panel-default') and .//h3[contains(text(), '맞은 문제')]]//a")
        pbar.set_postfix_str(f'userName: {user_name}, num solved: {len(solved_problems)}')
        for solved in solved_problems:
            interaction.append([user_name, solved.text, 1])


    time.sleep(3)

driver.quit()

# -- csv 파일로 저장
column_name = ['userName', 'problemId', 'solved']
gachon_user_df = pd.DataFrame(data=interaction, columns=column_name)
gachon_user_df.to_csv("./gachon_user_information.csv", index=False)

In [7]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
from tqdm.notebook import tqdm
import pandas as pd

# 크롬드라이버 옵션 (창 안보이게 숨기기)
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('headless')
driver = webdriver.Chrome(options=driver_option) 
driver.implicitly_wait(3)

try:
    # 크롬드라이버 실행
    driver = webdriver.Chrome(options=driver_option) 
    driver.implicitly_wait(3)

    gachon_user_tier = []

    driver.get('https://www.acmicpc.net/school/ranklist/187')
    time.sleep(3)

    page_elem = driver.find_elements(by=By.XPATH, value="(//ul[@class='pagination']/li/a)[position()>1]")
    MAX_PAGE = max([int(elem.get_attribute('href').split("/")[-1]) for elem in page_elem])


    for page in tqdm(range(MAX_PAGE)):
        driver.get(f'https://www.acmicpc.net/school/ranklist/187/{page + 1}')
        time.sleep(3)

        gachon_user_elem = driver.find_elements(by=By.CSS_SELECTOR, value='#ranklist > tbody > tr > td:nth-child(2) > a')
        user_list = [user_elem.get_attribute('href').split("/")[-1] for user_elem in gachon_user_elem] # 현재 page의 가천대학교 학생 목록

        pbar = tqdm(user_list)
        for user_name in pbar:
            driver.get(f'https://www.acmicpc.net/user/{user_name}')  # 백준 유저 정보 페이지 접속
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//*[@class='page-header']/h1"))
            )

            try:
                solvedac_tier_img_tag = driver.find_element(by=By.XPATH, value="//*[@class='page-header']/h1/a/img")
                solvedac_tier_tag_src = solvedac_tier_img_tag.get_attribute('src').split("/")[-1]
                solvedac_tier_num = solvedac_tier_tag_src.split(".")[0]
                pbar.set_postfix(name=user_name, tier=solvedac_tier_num, refresh=True)
                gachon_user_tier.append([user_name, solvedac_tier_num])
            except:
                gachon_user_tier.append([user_name, 0])

finally:
    driver.quit()

# -- csv 파일로 저장
column_name = ['user_id', 'tier']
gachon_user_df = pd.DataFrame(data=gachon_user_tier, columns=column_name)
gachon_user_df.to_csv("./gachon_user_tier.csv", index=False)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [14]:
gachon_user_df.loc[gachon_user_df['tier'] == '-1', 'tier'] = '0'
gachon_user_df.to_csv("./gachon_user_tier.csv", index=False)

In [15]:
gachon_user_df['tier'].unique()

array(['21', '22', '14', '20', '17', '18', '13', '16', '24', 0, '15',
       '12', '10', '11', '9', '7', '6', '8', '0', '5', '4', '3', '2', '1'],
      dtype=object)